In [1]:
import networkx as nx
import pandas
from hatchet import *
import os
import matplotlib.pyplot as plt
import platform

In [2]:
# libraries for gromov distance computation.
import numpy as np
import scipy as sp
import ot
from sklearn.decomposition import PCA
from sklearn import manifold

In [3]:
# from groupBy import groupBy
from state import State
from callgraph import CallGraph


In [4]:
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.2f}'.format
plt.rcParams['figure.figsize'] = (16, 12)

In [5]:
# Linux path
if platform.system() == "Linux":
	callflow_path = "/home/vidi/Work/llnl/CallFlow/"
else:
	#Mac OSx path
	callflow_path = "/Users/jarus/ucd/Research/Visualisation/projects/CallFlow"

dataset_path = ["data/lulesh-1/db-ampi4-100-1", "data/lulesh-1/db-ampi4-100-8"]
dataset = ['db-ampi4-100-1', 'db-ampi4-100-8']  

In [6]:
# Create Graphframes.
def create_gfs(file_format, paths):    
	print("Creating graphframes....")                                                                                             
	ret = []                                                                                                                         
	for idx, path in enumerate(paths):
		path = os.path.abspath(os.path.join(callflow_path, path)) 
		gf = GraphFrame()   
		gf.from_hpctoolkit(path)                                                                            
		ret.append(gf) 
		print(str(idx) + ":" + path)                                                                                              
	return ret 

In [7]:
# util functions
def lookup(df, node):                                                                                                                    
	return df.loc[df['node'] == node] 

def lookup_with_name(df, name):
	return df.loc[df['name'] == name]

def getMaxIncTime(gf):                                                                                                                   
	ret = 0.0                                                                                                                            
	for root in gf.graph.roots:                                                                                                          
		ret = max(ret, lookup(gf.dataframe, root)['time (inc)'].max())                                                           
	return ret                                                                                                                           
																																		 
def getMaxExcTime(gf):                                                                                                                   
	ret  = gf.dataframe['CPUTIME (usec) (E)'].max()                                                                                      
	return ret                                                                                                                           
			   
def special_lookup(gf, df_index):   
	return gf.dataframe.loc[gf.dataframe['name'] == df_index] 

In [8]:
# Filter graphframe and graph
def filter_gfs(gfs, filterBy):                                                                                                   
	# Create the graph frames from the paths and corresponding format using hatchet                                                  
	fgfs = []                                                                                                                        
	# Filter graphframes based on threshold                                                                                          
	for idx, gf in enumerate(gfs):                                                                                              
		print("Filtering the dataframe!")                                                                                         
		if filterBy == "IncTime":                                                                                          
			max_inclusive_time = getMaxIncTime(gf)                                                                             
			filter_gf = gf.filter(lambda x: True if(x['time (inc)'] > 0.01*max_inclusive_time) else False)                   
		elif self.args.filterBy == "ExcTime":                                                                                        
			max_exclusive_time = getMaxExcTime(gf)                                                                             
			print('[Filter] By Exclusive time = {0})'.format(max_exclusive_time))                                                 
			filter_gf = gf.filter(lambda x: True if (x['time'] > 0.01*max_exclusive_time) else False)                  
		else:                                                                                                                        
			print("Not filtering.... Can take forever. Thou were warned")                                                         
			filter_gf = gf                                                                                                           
		print('[Filter] Removed {0} rows.)'.format(gf.dataframe.shape[0] - filter_gf.dataframe.shape[0]))                                                                                                                            
		print("Grafting the graph!")                                                                                            
		filter_gf = filter_gf.squash()                                                                                                
		print("[Graft] {0} rows left".format(filter_gf.dataframe.shape[0])) 
		fgfs.append(filter_gf)                                              
	print(fgfs)
	return fgfs

In [9]:
# add n_index to the dataframe.
def add_n_index(gf):
	gf.dataframe['n_index'] = gf.dataframe.groupby('nid').ngroup()

def df_index_name_mapper(graph, df):
	ret = {}
	node_count = 0
	root = graph.roots[0]
	node_gen = graph.roots[0].traverse()
	try:
		while root.callpath != None:
			node_count += 1
			root = next(node_gen)
			ret[root.callpath[-1]] = root.df_index
	except StopIteration:
		pass
	finally:
		print("Total nodes in graph: ", node_count)
		del root
	return ret

# add df_index to the dataframe
def add_df_index(gf):
	df_index_name_map = df_index_name_mapper(gf.graph, gf.dataframe)
	gf.dataframe['df_index'] = gf.dataframe['name'].apply(lambda node: df_index_name_map[node] if node in df_index_name_map else 'as ')   

In [10]:
# add callee and caller data into the dataframe
def add_callers_and_callee(graph, df):
    callees = {}
    callers = {}
    root = graph.roots[0]
    node_gen = graph.roots[0].traverse()
    root_df = root.callpath[-1]
    callers[root_df] = []
    callees[root_df] = []
    try:                                                                                                                        
        while root.callpath != None:                                                                                            
            root = next(node_gen)                                                                                               
            if root.parents:                                                                                                     
                for idx, parent in enumerate(root.parents):
                    root_df = root.callpath[-1]                                                                                     
                    parent_df = parent.callpath[-1]                                                                            
                    if parent_df not in callees:                                                                                    
                        callees[parent_df] = []              
                    callees[parent_df].append(root_df)                                                                              
                    
                    if root_df not in callers:                                                                                      
                        callers[root_df] = []                                                                                       
                    callers[root_df].append(parent_df)                                                                              	
    except StopIteration:                                                                                                       
        pass                                                                                                                    
    finally:                                                                                                                    
        del root                                                                                                                
    
    df['callees'] = df['name'].apply(lambda node: callees[node] if node in callees else [])                           
    df['callers'] = df['name'].apply(lambda node: callers[node] if node in callers else []) 

In [11]:
#pre-process dataframe to add more information. 
def preprocess(state):
    preprocess = PreProcess.Builder(state).add_df_index().add_n_index().add_mod_index().add_callers_and_callees().add_show_node().add_vis_node_name().update_module_name().clean_lib_monitor().build() 

#NetworkX stuff.
def create_nx_graph(state):
	g = nx.DiGraph()
	return g

In [12]:
def main(dataset_path):
	dataset = []
	for idx, path in enumerate(dataset_path):
		dataset.append(path.split('/')[0])

	gfs = create_gfs('hpctoolkit', dataset_path)
	# filtered graph frames.
	fgfs = filter_gfs(gfs, 'IncTime')  
	return fgfs


In [13]:
dataset_path = ["data/hpctoolkit-lulesh-2"]
fgfs = main(dataset_path)

Creating graphframes....
0:/home/vidi/Work/llnl/CallFlow/data/hpctoolkit-lulesh-2
Filtering the dataframe!
[Filter] Removed 1204 rows.)
Grafting the graph!


/home/vidi/Work/llnl/CallFlow/src/hatchet/hatchet/graphframe.py:176: FutureWarning: 'node' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  num_rows_df = len(self.dataframe.groupby(['node']))
/home/vidi/Work/llnl/CallFlow/src/hatchet/hatchet/graphframe.py:268: FutureWarning: 'rank' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  agg_df = new_dataframe.groupby(index_names).agg(agg_dict)


[Graft] 1274 rows left


In [19]:
def process():
    states = []
    for idx, fgf in enumerate(fgfs):
        print("Shape of the dataframe from graph ({0}): {1}".format(dataset[idx], fgf.dataframe.shape))
        state = State(fgf)
        add_callers_and_callee(state.graph, state.df)
#         preprocess(state)
#         groupBy(state, 'module')
        print(state.df)
        states.append(state)
    return states

In [20]:
states = process()

Shape of the dataframe from graph (db-ampi4-100-1): (1274, 12)
                          time (inc)  time      nid  rank  \
node                rank                                    
<program root>      0            nan  0.00     0.00  0.00   
main                0            nan  0.00     1.00  0.00   
2809:MPI_Finalize   0            nan  0.00 1,233.00  0.00   
PMPI_Finalize       0            nan  0.00 1,234.00  0.00   
294:MPID_Finalize   0            nan  0.00 1,249.00  0.00   
...                              ...   ...      ...   ...   
<unknown procedure>              nan   nan      nan   nan   
<unknown file>:0                 nan   nan      nan   nan   
                                 nan   nan      nan   nan   
interp.c:0                       nan   nan      nan   nan   
__GI_sched_yield                 nan   nan      nan   nan   

                                                                       file  \
node                rank                                        

In [ ]:
def write_graph():
    for 

In [ ]:
aggr_times = {}
for idx, state in enumerate(states):
    pivot = pd.pivot_table(state.df, values=['CPUTIME (usec) (I)'], index=['module'], columns=['rank'])
    for idx, row in enumerate(pivot.iterrows()):
        if(row[0] not in aggr_times):
            aggr_times[row[0]] = []
        aggr_times[row[0]].append(row[1].mean())
print(aggr_times)

In [ ]:
props = {
    "lulesh2.0" :{
        "name" : "lulesh",
        "files" : [ "mpi-linux-x86_64-ifort-mpicxx/tmp/", "mpi-linux-x86_64-ifort-mpicxx/", "mpi-linux-x86_64-ifort-mpicxx/tmp/libs/ck-libs/", "mpi-linux-x86_64-ifort-mpicxx/tmp/libs/conv-libs/"],
        "functions" : [] 
    },
    "libmpi.so.12.0.5" :{
        "name" : "MPI",
        "files" : ["../src/mpi/"],
        "functions" : []
    }
}

In [ ]:
def groupby(df, keys, metric = 'mean'):
    # Groups data by the keys provided
    groups = df.groupby(keys)
    measure = getattr(groups, metric)
    data = measure() 
    return data

In [ ]:
df = states[0].df

def rename_module_names(df, props):
    module_names = list(props.keys())
    for module_name in module_names:
        rename_to = props[module_name]['name']
        files = props[module_name]['files']
        function = props[module_name]['functions']
        for idx, row in df.iterrows():
            if row.module:
                if(row.module == module_name):
                    df.set_value(idx, 'module', rename_to)
                else:
                    pass
                
        print(df['file'].str.contains(files[0], regex=True))

    print(df['module'].unique().tolist())


rename_module_names(df, props)
print(df[df['module']=='Unkno']['name'].unique().tolist())
group_df = groupby(df, ['name', 'rank'])
print(group_df)
for idx, row in df.iterrows():
    pass

In [ ]:
# Joint plot of the Inclusive and Exclusive times.
df = states[0].df
df2 = states[1].df
root_max_df = 107348942.00
root_max_df2 = 145114370.00
sns.jointplot('time (inc)', 'time',
              df.loc[(df['time (inc)'] < 0.10*107348942.00) &
                     (df['time'] > 0)],
              alpha=.25, marker='.', height=8);
sns.jointplot('time (inc)', 'time',
              df2.loc[(df2['time (inc)'] < 0.10*145114370.00) &
                     (df2['time'] > 0)],
              alpha=.25, marker='.', height=8);